In [ ]:
import os
os.chdir('..')
import model_database_api as api
print(os.getcwd())

import matplotlib.pyplot as plt

from DLBio.helpers import to_uint8_image
import numpy as np
# annoying plt warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#model_folder_name = 'CifarJOVFPNet_N9_s744'
#IS_CIFAR = True

model_folder_name = 'resnet50_imagenet_layer_start_q1'
IS_CIFAR = False

df = api.get_dataframe(model_folder_name)


## Do the low-entropy maps look plausible?

In [ ]:
def compute_entropy(image):
    if image.dtype != 'uint8':
        image = to_uint8_image(image)
    
    counts, _ = np.histogram(image.flatten(), bins=list(range(256)))
    dist = counts / counts.sum()
    return -1.* (dist * np.log2(dist + 1e-6)).sum()
    

In [ ]:
if True:
    X = api.get_activation_examples_input(IS_CIFAR)
    operation = 'res'
    tmp = api.get_subtable(df, {'position':0})
    tmp = tmp[tmp['ent_per_fmap-'+operation] > 0.2]
    tmp = tmp.sort_values('ent_per_fmap-'+operation, ascending=True)

    ctr = 0

    for _, row in tmp.iterrows():
        act = api.get_activations(row, operation)
        for i in [0, 1,2,3,4]:
            ent = compute_entropy(act[i,...])
            _, ax = plt.subplots(1,2)
            ax[0].imshow(to_uint8_image(act[i,...]))
            ax[1].imshow(to_uint8_image(X[i,...]))
            ax[0].set_title(
                ['stat: ', round(row['ent_per_fmap-'+operation],2),
                'here: ', round(ent,2)]
            )
            ax[1].set_title(round(row['kd_angles'],2))

            ctr += 1

        if ctr > 30:
            break

## Do the degofes maps look plausible?

In [ ]:
if True:
    X = api.get_degree_of_es_input(+1,IS_CIFAR)
    operation = 'mult'
    tmp = api.get_subtable(df, {'position':0})
    tmp = api.get_degree_of_es_analysis(tmp, ['mult'])
    #tmp = tmp[tmp[operation+'-id2_pos'] > .1]
    #tmp = tmp[tmp[operation+'-id1_pos'] < 1.]
    #tmp = tmp.sort_values(operation+'-id2_pos', ascending=False)

    ctr = 0

    for _, row in tmp.iterrows():
        act = api.get_deg_of_es_activation(row, operation)
        _, ax = plt.subplots(1,2)
        ax[0].imshow(to_uint8_image(act[0,...]))
        ax[1].imshow(to_uint8_image(X))
        ax[0].set_title(round(row[operation+'-id_ratio'],2))
        ax[1].set_title(round(row['kd_angles'],2))

        ctr += 1

        if ctr > 30:
            break